In [1]:
# This script performs analysis on gaze state classification

# Plan: 

# metrics that don't distiguish between remote / collocated

#  - proportion of time looking at other participants
#  - proportion of time transitioning
#  - proportion of time averting
#  - glance duration
#  - glance frequency
#  - transition duration

# metrics that are person dependent

# - proportion of time looking at each other participant
# - frequency of glances at other participants
# - proportion of mutual gaze with other participants
# - length of mutual gazes with other participants
# - frequency of mutual gazes with other participants


In [2]:
import re
import matplotlib.pyplot as plt
from enum import Enum
import os
import statistics

In [3]:
#define location of data to process

# data_dir="C:/Users/gary4/Documents/repos/Recording-And-Analysis-Plugin/Data/group0"
# data_dir="Z:/Gary/Research/APlausE-MR_MR4_CollaborativeTelepresenceStudy/gazeAnalysis/group0"
data_dir="Z:/builds/aplause-mr/UNITY_APP/aplause-mr-20240707-studystate-v1/gazeAnalysis"

# find all "gaze_classification_results" files
files = os.listdir(data_dir)
files_to_process = [f for f in files if "gaze_classification_results.csv" in f] 

print("Found " + str(len(files_to_process)) + " files to process")


Found 64 files to process


In [4]:
# define some constants

NUM_PARTICIPANTS = 4


In [5]:
class GazeState(Enum):
    TRANSITION = 0
    LOCAL = 1
    REMOTE_ADJACENT = 2
    REMOTE_OPPOSITE = 3
    AVERTED = 4
    
# Define colors for each GazeState
gaze_state_colors = {
    GazeState.TRANSITION: 'lightgray',
    GazeState.LOCAL: 'limegreen',
    GazeState.REMOTE_ADJACENT: 'yellow',
    GazeState.REMOTE_OPPOSITE: 'cornflowerblue',
    GazeState.AVERTED: 'black',
}

In [32]:
# define some data structures for reading data

class GazeClassification:

    def __init__(self, desc):
        self.times = []
        self.values = []
                
        # find participant ID for gazer
        participant_label_ends = [m.end() for m in re.finditer('Participant', desc)]
        self.gazer = int(desc[participant_label_ends[0]])
        
    def add_value(self,time,value):
        self.times.append(time)
        self.values.append(GazeState(value))
        
    def GetGazeStateTotalDurations(self):
        
        state_durations = [0] * len(GazeState)
        
        for i in range(1, len(self.times)):

            duration = self.times[i] - self.times[i - 1]
            state = self.values[i - 1]
            state_durations[state.value] += duration
            
        return state_durations
    
    def GetGazeStateDurationList(self):
        
#         duration_lists = [[]]*len(GazeState)
        duration_lists = {state: [] for state in GazeState}

        last_state = self.values[0]
        state_start = self.times[0]
        
        for i in range(1, len(self.times)):
            
            state = self.values[i]
            
            if state != last_state:
                duration_lists[last_state].append( self.times[i] - state_start )
                state_start = self.times[i]
            
            last_state = state
    
        return duration_lists


In [33]:
# data structure for writing data

class GazeAnalysisResult:
    
    def __init__(self, rec_file_name, participant_id):
        
        self.participant_id = participant_id
        
        # TODO determine group and trial from file name
        self.trial = 0
        self.group = 0
        
        # TODO determine conditions
        self.avatar_condition = 0
        self.audio_condition = 0
        
        

In [34]:
def ReadData(file_name):
    file_path = data_dir + "/" + file_name
    print("Processing file: " + file_path)

    classification_streams = []

    f = open(file_path, "r")

    while True:
        line = f.readline()
        if not line:
            break

        if line.startswith("AnalysisQuery"):
            classification_streams.append(GazeClassification(line))
            
        elif 'Threshold' in line or line.startswith('Time') or not line.strip():
            pass

        else:
            in_data = line.split(',')

            classification_streams[-1].add_value(float(in_data[0]), int(in_data[1]))

    f.close()
    return classification_streams
    



In [35]:

#  - proportion of time looking at other participants
#  - proportion of time transitioning
#  - proportion of time averting
#  - glance duration
#  - transition duration
#  - glance frequency


In [48]:
def ProcessFile(file_name):
    
    classification_streams = ReadData(file_name)

    if len(classification_streams) != NUM_PARTICIPANTS:
        raise RuntimeError("Found incorrect number of gaze classification streams")
    
    for p in range(NUM_PARTICIPANTS):
        result_struct = GazeAnalysisResult(file_name, p)

        trial_length = classification_streams[p].times[-1]
        
        # get duration for which each state is held
        state_durations = classification_streams[p].GetGazeStateTotalDurations()
        state_duration_list = classification_streams[p].GetGazeStateDurationList()
        
        #  - proportion of time looking at other participants
        gaze_others_pc = (state_durations[GazeState.LOCAL.value] \
                          + state_durations[GazeState.REMOTE_ADJACENT.value] \
                          + state_durations[GazeState.REMOTE_OPPOSITE.value]) \
                            / trial_length * 100.0;

        #  - proportion of time transitioning
        gaze_transition_pc = 100.0 * state_durations[GazeState.TRANSITION.value] / trial_length;
        
        #  - proportion of time averting        
        gaze_avert_pc = 100.0 * state_durations[GazeState.AVERTED.value] / trial_length;

        # - transition duration
        gaze_transition_mean_duration_sec = statistics.mean(state_duration_list[GazeState.TRANSITION])
    
        # - glance duration
        all_glance_durations = state_duration_list[GazeState.LOCAL] \
                                + state_duration_list[GazeState.REMOTE_ADJACENT] \
                                + state_duration_list[GazeState.REMOTE_OPPOSITE]
        gaze_glance_mean_duration_sec = statistics.mean(all_glance_durations)
        
        #  - glance frequency
        glance_per_min = len(all_glance_durations) / trial_length * 60
        
        
        result_struct.gaze_others_pc = gaze_others_pc
        result_struct.gaze_transition_pc = gaze_transition_pc
        result_struct.gaze_avert_pc = gaze_avert_pc
        result_struct.gaze_transition_mean_duration_sec = gaze_transition_mean_duration_sec
        result_struct.gaze_glance_mean_duration_sec = gaze_glance_mean_duration_sec
        result_struct.glance_per_min = glance_per_min
        
        
        # TODO analysis for participant p
        
    
    
    
    return result_struct
            

In [49]:

ProcessFile(files_to_process[0])




Processing file: Z:/builds/aplause-mr/UNITY_APP/aplause-mr-20240707-studystate-v1/gazeAnalysis/aplausemr_group0_date20240708_10_42_trial0_experimentcontroller_gaze_classification_results.csv
